In [6]:
import requests
from bs4 import BeautifulSoup
import json


In [7]:
import requests
from bs4 import BeautifulSoup
import json

# URL страницы
url = 'https://www.nbkr.kg/index1.jsp?item=2746&lang=RUS&begin_day=25&begin_month=09&begin_year=2023&end_day=01&end_month=10&end_year=2024'

# Заголовки для имитации браузера
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) ' \
                  'AppleWebKit/537.36 (KHTML, like Gecko) ' \
                  'Chrome/58.0.3029.110 Safari/537.3'
}

# Отправляем GET-запрос к странице
response = requests.get(url, headers=headers)

# Проверяем успешность запроса
if response.status_code != 200:
    print(f'Ошибка при запросе страницы: {response.status_code}')
    exit()

# Парсим HTML с помощью BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Получаем полный HTML как строку
html_content = str(soup)

In [8]:
from bs4 import BeautifulSoup
import json

full_html = f'''
<table class="content-table">
<tr align="center" bgcolor="#2D4283">     
    <td class="informer_header_large" style="color: #ffffff">Дата</td>     
    <td class="informer_header_large" style="color: #ffffff">Мерные золотые слитки, г.</td>     
    <td class="informer_header_large" style="color: #ffffff">Цена обратного выкупа, сом</td>     
    <td class="informer_header_large" style="color: #ffffff">Цена продажи, сом</td>  
</tr>
{html_content}
</table>
'''

soup = BeautifulSoup(full_html, 'html.parser')

table = soup.find('table', class_='content-table')
if not table:
    print('Таблица не найдена в предоставленном HTML.')
    exit()

headers = []
header_row = table.find('tr')
if not header_row:
    print('Строка заголовков не найдена в таблице.')
    exit()

for th in header_row.find_all(['td', 'th']):
    header_text = th.get_text(strip=True)
    headers.append(header_text)

if not headers:
    print('Заголовки столбцов не найдены.')
    exit()

data = []
for row in table.find_all('tr')[1:]:  
    cols = row.find_all('td')
    if len(cols) != len(headers):
        print('Пропуск строки из-за несоответствия количества столбцов.')
        continue 
    row_data = {}
    for header, col in zip(headers, cols):
        cell_text = col.get_text(strip=True)
        row_data[header] = cell_text
    data.append(row_data)

json_data = json.dumps(data, ensure_ascii=False, indent=4)

with open('table_data.json', 'w', encoding='utf-8') as f:
    f.write(json_data)

print('Данные таблицы успешно сохранены в "table_data.json"')


Данные таблицы успешно сохранены в "table_data.json"


In [9]:
import pandas as pd
json_data = pd.read_json('table_data.json')
df = pd.DataFrame(json_data)
df


,Дата,"Мерные золотые слитки, г.","Цена обратного выкупа, сом","Цена продажи, сом"
0,Дата,"Мерные золотые слитки, г.","Цена обратного выкупа, сом","Цена продажи, сом"
1,01.10.2024,1.00,8 507.50,8 550.00
2,01.10.2024,2.00,15 738.50,15 801.50
3,01.10.2024,5.00,37 440.50,37 552.50
4,01.10.2024,10.00,73 101.50,73 247.50
...,...,...,...,...
1484,02.10.2023,2.00,12 185.50,12 234.00
1485,02.10.2023,5.00,28 501.50,28 587.00
1486,02.10.2023,10.00,56 094.00,57 484.50
1487,02.10.2023,31.1035,171 615.50,174 727.50


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

# Функция для генерации URL с заданным диапазоном дат
def generate_url(begin_day, begin_month, begin_year, end_day, end_month, end_year):
    base_url = 'https://www.nbkr.kg/index1.jsp'
    params = {
        'item': '2746',
        'lang': 'RUS',
        'begin_day': str(begin_day).zfill(2),
        'begin_month': str(begin_month).zfill(2),
        'begin_year': str(begin_year),
        'end_day': str(end_day).zfill(2),
        'end_month': str(end_month).zfill(2),
        'end_year': str(end_year)
    }
    # Преобразуем параметры в строку запроса
    query_string = '&'.join([f"{key}={value}" for key, value in params.items()])
    return f"{base_url}?{query_string}"

# Заголовки для имитации браузера
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) ' \
                  'AppleWebKit/537.36 (KHTML, like Gecko) ' \
                  'Chrome/58.0.3029.110 Safari/537.3'
}

# Список лет для парсинга
years = range(2020, 2025)  # 2020 по 2024 включительно

# Инициализируем пустой DataFrame для объединения данных
combined_df = pd.DataFrame()

for year in years:
    print(f"Парсинг данных за {year} год...")
    
    # Задаем начальную и конечную даты для текущего года
    begin_day = 1
    begin_month = 1
    begin_year = year
    
    # Для 2024 года конечная дата — текущая дата
    if year == 2024:
        today = datetime.today()
        end_day = today.day
        end_month = today.month
        end_year = today.year
    else:
        end_day = 31
        end_month = 12
        end_year = year
    
    # Генерируем URL для текущего года
    url = generate_url(begin_day, begin_month, begin_year, end_day, end_month, end_year)
    print(f"URL: {url}")
    
    # Отправляем GET-запрос к странице
    response = requests.get(url, headers=headers)
    
    # Проверяем успешность запроса
    if response.status_code != 200:
        print(f'Ошибка при запросе страницы за {year} год: {response.status_code}')
        continue  # Переходим к следующему году
    
    # Парсим HTML с помощью BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Предполагаем, что данные находятся в таблице <table>
    table = soup.find('table')
    
    if not table:
        print(f'Таблица данных не найдена за {year} год.')
        continue  # Переходим к следующему году
    
    # Извлекаем заголовки таблицы
    headers_table = []
    header_row = table.find('tr')
    for th in header_row.find_all('th'):
        headers_table.append(th.get_text(strip=True))
    
    # Извлекаем строки таблицы
    rows = []
    for tr in table.find_all('tr')[1:]:  # Пропускаем заголовок
        cells = tr.find_all(['td', 'th'])
        row = [cell.get_text(strip=True) for cell in cells]
        if row:
            rows.append(row)
    
    # Создаем DataFrame для текущего года
    df_year = pd.DataFrame(rows, columns=headers_table)
    
    # Добавляем столбец года, если его нет
    if 'Год' not in df_year.columns:
        df_year['Год'] = year
    
    # Объединяем с общим DataFrame
    combined_df = pd.concat([combined_df, df_year], ignore_index=True)
    print(f"Данные за {year} год успешно добавлены.\n")

# Сохраняем объединенные данные в CSV файл
combined_df.to_csv('combined_data_2020_2024.csv', index=False, encoding='utf-8-sig')
print("Все данные успешно сохранены в 'combined_data_2020_2024.csv'.")


Парсинг данных за 2020 год...
URL: https://www.nbkr.kg/index1.jsp?item=2746&lang=RUS&begin_day=01&begin_month=01&begin_year=2020&end_day=31&end_month=12&end_year=2020


ValueError: 0 columns passed, passed data had 4 columns

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

# Функция для генерации URL с заданным диапазоном дат
def generate_url(begin_day, begin_month, begin_year, end_day, end_month, end_year):
    base_url = 'https://www.nbkr.kg/index1.jsp'
    params = {
        'item': '2746',
        'lang': 'RUS',
        'begin_day': str(begin_day).zfill(2),
        'begin_month': str(begin_month).zfill(2),
        'begin_year': str(begin_year),
        'end_day': str(end_day).zfill(2),
        'end_month': str(end_month).zfill(2),
        'end_year': str(end_year)
    }
    # Преобразуем параметры в строку запроса
    query_string = '&'.join([f"{key}={value}" for key, value in params.items()])
    return f"{base_url}?{query_string}"

# Заголовки для имитации браузера
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) ' \
                  'AppleWebKit/537.36 (KHTML, like Gecko) ' \
                  'Chrome/58.0.3029.110 Safari/537.3'
}

# Список лет для парсинга
years = range(2010, 2025)  # 2020 по 2024 включительно

# Инициализируем пустой DataFrame для объединения данных
combined_df = pd.DataFrame()

for year in years:
    print(f"Парсинг данных за {year} год...")
    
    # Задаем начальную и конечную даты для текущего года
    begin_day = 1
    begin_month = 1
    begin_year = year
    
    # Для 2024 года конечная дата — текущая дата
    if year == 2024:
        today = datetime.today()
        end_day = today.day
        end_month = today.month
        end_year = today.year
    else:
        end_day = 31
        end_month = 12
        end_year = year
    
    # Генерируем URL для текущего года
    url = generate_url(begin_day, begin_month, begin_year, end_day, end_month, end_year)
    print(f"URL: {url}")
    
    # Отправляем GET-запрос к странице
    response = requests.get(url, headers=headers)
    
    # Проверяем успешность запроса
    if response.status_code != 200:
        print(f'Ошибка при запросе страницы за {year} год: {response.status_code}')
        continue  # Переходим к следующему году
    
    # Парсим HTML с помощью BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Предполагаем, что данные находятся в таблице <table>
    table = soup.find('table')
    
    if not table:
        print(f'Таблица данных не найдена за {year} год.')
        continue  # Переходим к следующему году
    
    # Проверка наличия строк в таблице
    rows_in_table = table.find_all('tr')
    if len(rows_in_table) < 2:
        print(f'Таблица за {year} год пуста или имеет недостаточно строк.')
        continue
    
    # Вывод HTML таблицы для отладки (можно закомментировать после проверки)
    # print(f"HTML таблицы за {year} год:")
    # print(table.prettify()[:1000])
    
    # Извлекаем заголовки таблицы
    headers_table = []
    header_row = table.find('tr')
    if header_row:
        for th in header_row.find_all(['th', 'td']):
            headers_table.append(th.get_text(strip=True))
    else:
        print(f'Заголовок таблицы не найден за {year} год.')
        continue
    
    # Извлекаем строки таблицы
    rows = []
    for tr in table.find_all('tr')[1:]:  # Пропускаем заголовок
        cells = tr.find_all(['td', 'th'])
        row = [cell.get_text(strip=True) for cell in cells]
        if row:
            rows.append(row)
    
    # Проверка наличия данных
    if not rows:
        print(f'Нет данных для добавления за {year} год.')
        continue
    
    # Проверка соответствия количества колонок
    if len(headers_table) != len(rows[0]):
        print(f'Несоответствие количества колонок в заголовках и данных за {year} год.')
        continue
    
    # Создаем DataFrame для текущего года
    try:
        df_year = pd.DataFrame(rows, columns=headers_table)
    except Exception as e:
        print(f'Ошибка при создании DataFrame за {year} год: {e}')
        continue
    
    # Добавляем столбец года, если его нет
    if 'Год' not in df_year.columns:
        df_year['Год'] = year
    
    # Объединяем с общим DataFrame
    combined_df = pd.concat([combined_df, df_year], ignore_index=True)
    print(f"Данные за {year} год успешно добавлены.\n")

# Проверка наличия данных перед сохранением
if not combined_df.empty:
    # Сохраняем объединенные данные в CSV файл
    combined_df.to_csv('gold_data_2010_2024.csv', index=False, encoding='utf-8-sig')
    print("Все данные успешно сохранены в 'combined_data_2020_2024.csv'.")
else:
    print("Нет данных для сохранения.")


Парсинг данных за 2010 год...
URL: https://www.nbkr.kg/index1.jsp?item=2746&lang=RUS&begin_day=01&begin_month=01&begin_year=2010&end_day=31&end_month=12&end_year=2010
Таблица за 2010 год пуста или имеет недостаточно строк.
Парсинг данных за 2011 год...
URL: https://www.nbkr.kg/index1.jsp?item=2746&lang=RUS&begin_day=01&begin_month=01&begin_year=2011&end_day=31&end_month=12&end_year=2011
Таблица за 2011 год пуста или имеет недостаточно строк.
Парсинг данных за 2012 год...
URL: https://www.nbkr.kg/index1.jsp?item=2746&lang=RUS&begin_day=01&begin_month=01&begin_year=2012&end_day=31&end_month=12&end_year=2012
Таблица за 2012 год пуста или имеет недостаточно строк.
Парсинг данных за 2013 год...
URL: https://www.nbkr.kg/index1.jsp?item=2746&lang=RUS&begin_day=01&begin_month=01&begin_year=2013&end_day=31&end_month=12&end_year=2013
Таблица за 2013 год пуста или имеет недостаточно строк.
Парсинг данных за 2014 год...
URL: https://www.nbkr.kg/index1.jsp?item=2746&lang=RUS&begin_day=01&begin_mont

In [7]:
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import json
from demir import demir
from mbank import mbank
from optima import optima
import requests

def main():
    demir_url = 'https://demirbank.kg/ru'
    mbank_url = 'https://mbank.kg/'
    optima_url = 'https://www.optimabank.kg/index.php?option=com_nbrates&view=default&lang=ru'

    demir_df = demir(demir_url)
    mbank_df = mbank(mbank_url)
    optima_df = optima(optima_url) # u can give two variables (url = url, dates = [list of date]) default is current date

    demir_df.rename(columns={'currency': 'Валюта', 'buy': 'Покупка', 'sell': 'Продажа', 'date': 'Дата', 'bank_name': 'Банк', 'type': 'Курс'}, inplace=True)
    mbank_df.rename(columns={'currency': 'Валюта', 'buy': 'Покупка', 'sell': 'Продажа', 'date': 'Дата', 'bank_name': 'Банк', 'type': 'Курс'}, inplace=True)
    optima_df.rename(columns={'currency': 'Валюта', 'buy': 'Покупка', 'sell': 'Продажа', 'date': 'Дата', 'bank_name': 'Банк', 'type': 'Курс'}, inplace=True)

    combined_df = pd.concat([demir_df, mbank_df, optima_df], ignore_index=True)
 
    today_date = datetime.now().strftime('%Y-%m-%d') 
    filename = f'combined_data_{today_date}.xlsx'
    combined_df.to_excel(filename, index=False)
    print(f"Data saved to {filename}")

main()


  bank_name         type currency      buy     sell        date
0     Демир     Наличный      EUR  92.7000  93.7000  2024-10-08
1     Демир     Наличный      USD  84.8500  85.3500  2024-10-08
2     Демир     Наличный      RUB   0.8800   0.9200  2024-10-08
3     Демир     Наличный      TRY   2.3500   2.6100  2024-10-08
4     Демир     Наличный      KZT   0.1200   0.1900  2024-10-08
5     Демир  Безналичный      EUR  92.7000  93.7000  2024-10-08
6     Демир  Безналичный      USD  84.4500  84.9500  2024-10-08
7     Демир  Безналичный      RUB   0.8610   0.9400  2024-10-08
8     Демир  Безналичный      TRY   2.3500   2.6100  2024-10-08
9     Демир  Безналичный      KZT   0.1195   0.2066  2024-10-08


c:\Users\asus\MyProjects\BankProject\demir.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_nal = pd.read_html(str(tables[0]))[0]
c:\Users\asus\MyProjects\BankProject\demir.py:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_beznal = pd.read_html(str(tables[1]))[0]


NameError: name 'requests' is not defined

In [5]:
data = len(pd.read_csv('combined_data_2020_2024.csv'))
data

7062

In [6]:
df = len(pd.read_csv('gold_data_2010_2024.csv'))
df

15342